In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ratio = 0.2
cutoff = 100

rng = np.random.default_rng(0)

In [3]:
# Reading metadata
data = pd.read_csv('minimal_metadata.csv')

sites = data['primary_site'].unique().tolist()
site_counts = data.groupby('primary_site')['id'].count().to_dict()

# Creating a list to store which rows to download
download_ids = set()
no_download_ids = set(range(len(data)))

for site in sites:
    data_site = data[data['primary_site'] == site]
    
    if site_counts[site] <= cutoff:
        ids = set(data_site.index.to_list())
    else:
        num_download = int(np.floor(ratio * site_counts[site]))
        random_downloads = rng.integers(low=0, high=site_counts[site], size=num_download)
        ids = set(data_site.iloc[random_downloads].index.to_list())
        
    download_ids.update(ids)

no_download_ids.difference_update(download_ids)

download_ids = list(download_ids)
no_download_ids = list(no_download_ids)

assert len(download_ids) + len(no_download_ids) == len(data)

In [4]:
cols = ['id', 'filename', 'md5', 'size', 'state']
download_manifest = data[cols].iloc[download_ids].copy(deep=True)
no_download_manifest = data[cols].iloc[no_download_ids].copy(deep=True)

download_manifest.to_csv('download_manifest.txt', sep='\t', index=False)
no_download_manifest.to_csv('no_download_manifest.txt', sep='\t', index=False)

cols = ['id', 'filename', 'md5', 'size', 'state', 'primary_site']
download_metadata = data[cols].iloc[download_ids].copy(deep=True)
no_download_metadata = data[cols].iloc[no_download_ids].copy(deep=True)

download_metadata.to_csv('download_metadata.csv', index=False)
no_download_metadata.to_csv('no_download_metadata.csv', index=False)